                                             2. extracting companies

In [112]:
import ast
import numpy as np
import pandas as pd
import nltk
import itertools
import re

In [113]:
# load the corpus and the companies names csv

In [114]:
with open('all_news.txt','r') as an:
    corpus = ast.literal_eval(an.read())
assert len(corpus) == 730

In [115]:
corpus = list(itertools.chain(*corpus))

In [116]:
with open('train_set.txt','r') as tf:
    train_set = ast.literal_eval(tf.read())
with open('validate_set.txt','r') as vf:
    valid_set = ast.literal_eval(vf.read())
with open('test_set.txt','r') as tef:
    test_set = ast.literal_eval(tef.read())

In [117]:
def process_company(company):
    
    company = company.strip()
    
    if len(company) == 0:
        return False
    else:
        return company

In [118]:
company_path = 'all/companies.csv'

In [119]:
company_file = open(company_path, 'r', encoding ='latin-1').readlines()
companies = map(lambda x: process_company(x), company_file)
companies = list(set(companies))

In [120]:
# remove stopwords 

In [121]:
from nltk.corpus import stopwords

In [122]:
stop_words = stopwords.words('english')

In [123]:
def check_sw(name):
    for word in stop_words:
        if word in name.lower().split(): 
            return False

    return True
def remove_sw(name):
    modified_name = [ ]
    for word in name.split():
        if word.lower() not in stop_words:
            modified_name.append(word)
            
    return " ".join(modified_name)

In [124]:

def find_names(article):
    
    regulation = "(?:(?:[A-Z]+[a-z]*) ?)+"
    pattern = re.compile(regulation)
    
    names = map(lambda s: re.findall(pattern, s), article)
    names = filter(None, names)
    names = itertools.chain(*names)
    
    
    names = map(lambda n: n.strip(), names)
    names = filter(check_sw, names)
    names = filter(lambda i: len(i) >= 4, names)
    names = set(names)
    
    return names

In [125]:
companies = map(lambda x: remove_sw(x), companies)
companies = list(companies)

In [126]:
from nltk.tokenize import TweetTokenizer

In [132]:
def get_features(names, article):
    
    features =[]
    def check_in_sentence(name, sentence):
        return ((" " + name + " " in sentence)
             or (sentence[:len(name)+1] == name + " ")
             or (sentence[-len(name)+1:] == " " + name))
    
    for name in names:
        sentences = map(lambda x: check_in_sentence(name, x), article)
        
        for sentence in sentences:
            feature = {'name':name}
            
            name_sp = name.split(' ')
            first_word = name_sp[0]
            last_word = name_sp[-1]
            feature['first_name_length'] = len(first_word)
            feature['last_name_length'] = len(last_word)
            
            sen = re.sub(r"[^A-Za-z0-9 ]", " ", sentence)
            tk = TweetTokenizer()
            words = tk.tokenize(sen)
            first_word_index = words.index(first_word)
            last_word_index = words.index(last_word)
            
            pos_words = nltk.pos_tag(words)
            feature['first_word_tag'] = pos_words[first_word_index][1]
            feature['last_word_tag'] = pos_words[last_word_index][1]
            
            number_of_Cap = sum(1 for Cap in name if Cap.istitle())
            feature['number_of_Cap'] = number_of_Cap
            prob_of_Cap = number_of_Cap/len(name)
            feature['Prob_of_Cap'] = prob_of_Cap
            feature['is_company'] = int(name in companies)
            
            valuable_words = {'Company','Inc','Group','Corporation','Co','Corp','Capital','Management','Ltd'}
            
            for value_word in valueable_words:
                if value_word in name_sp:
                    feature["name_keyword"] = True
                    break
                else: 
                    feature["name_keyword"] = False
            
            features.append(feature)
    return features  

In [133]:
train =[]
for i, article in enumerate(train_set):
    names = find_names(article)
#     print(list(names))
    features = get_features(names,article)
    
    train.extend(features)
    
    if i % 500 == 0: 
        print(f"This program has processed {i} articles")
    
df_train = pd.DataFrame(train)

In [ ]:
import sklearn
from sklearn import metrics
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

In [134]:
df_train['is_company'].value_counts()

In [135]:
df_positive = df_train[df_train.is_company==1]
df_negative = df_train[df_train.is_company==0]
resample_class = len(df_positive) * 10
df_negative = resample(df_negative, replace = False, n_samples = resample_class)
df_positive = resample(df_positive, replace = True, n_samples = resample_class)
df_train2 = pd.concat([df_negative, df_positive])
df_train2.first_name_pos = pd.factorize(df_train2.first_word_pos)[0]
df_train2.last_name_pos = pd.factorize(df_train2.last_word_pos)[0]

In [136]:
RFC = RandomForestClassifier(n_estimators = 20)
featuress = [i for i in df_train2.columns if i not in ("is_company", "name")]
RFC.fit(df_train2[featuress], df_train2["is_company"])

In [ ]:
test = [ ]
for i, article in enumerate(test_set):
    names = find_names(article)
    features = get_features(names, article)
    test.extend(features)
    
    if i % 1000 == 0: print(f"The program has processed {i} articles")
    
dftest = pd.DataFrame(test)
dftest.first_name_pos = pd.factorize(dftest.first_word_pos)[0]
dftest.last_name_pos = pd.factorize(dftest.last_word_pos)[0]

In [ ]:
predict = RFC.predict(dftest[featuress])
get_metrics(dftest.is_ceo, predict)

In [ ]:
predt = RFC.predict(df_train2[featuress])
predv = RFC.predict(dfv[featuress])
preds = RFC.predict(dftest[featuress])


found_companiess = [ ]
found_companies.extend(list(df_train2.iloc[np.where(predt == 1)].name))
found_companies.extend(list(dfv.iloc[np.where(predv == 1)].name))
found_companies.extend(list(dftest.iloc[np.where(preds == 1)].name))


with open("found_companies.txt", "w") as f:
    for ceo in found_comanies:
        f.write(company)